In [ ]:
import json
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

class ActionAnalyzer:
    """分析JSONL格式的动作类型数据"""
    
    def __init__(self, jsonl_path):
        """
        初始化分析器
        
        Parameters:
        -----------
        jsonl_path : str
            JSONL文件路径
        """
        self.jsonl_path = jsonl_path
        self.data = []
        self.stats = None
        
    def load_data(self):
        """加载JSONL数据"""
        with open(self.jsonl_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    self.data.append(json.loads(line))
        print(f"✓ 成功加载 {len(self.data)} 条数据")
        return self
    
    def extract_action_type(self, id_string):
        """
        从ID中提取动作类型
        
        Parameters:
        -----------
        id_string : str
            格式: 数据集名称/动作类型/时间戳
            
        Returns:
        --------
        str : 动作类型
        """
        parts = id_string.split('/')
        if len(parts) >= 2:
            return parts[1]  # 返回两个斜杠之间的内容
        return None
    
    def analyze(self):
        """分析数据并生成统计信息"""
        action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
        
        for item in self.data:
            id_string = item.get('id', '')
            action_type = self.extract_action_type(id_string)
            
            if action_type:
                # 统计trajectory（完整的ID作为唯一标识）
                action_data[action_type]['trajectories'].add(id_string)
                # 统计样本数
                action_data[action_type]['samples'] += 1
        
        # 转换为DataFrame
        stats_list = []
        for action_type, info in action_data.items():
            stats_list.append({
                '动作类型': action_type,
                'Trajectory数量': len(info['trajectories']),
                '样本数量': info['samples']
            })
        
        self.stats = pd.DataFrame(stats_list).sort_values('样本数量', ascending=False)
        return self
    
    def print_summary(self):
        """打印统计摘要"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        print("\n" + "="*60)
        print("📊 动作类型统计分析")
        print("="*60)
        
        print(f"\n总计：")
        print(f"  • 动作类型数量: {len(self.stats)}")
        print(f"  • Trajectory总数: {self.stats['Trajectory数量'].sum()}")
        print(f"  • 样本总数: {self.stats['样本数量'].sum()}")
        
        print(f"\n详细统计：")
        print(self.stats.to_string(index=False))
        
        print("\n" + "="*60)
        
    def plot_statistics(self, figsize=(14, 6)):
        """绘制统计图表"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        fig, axes = plt.subplots(1, 2, figsize=figsize)
        
        # 图1：Trajectory数量
        axes[0].barh(self.stats['动作类型'], self.stats['Trajectory数量'], 
                     color='steelblue', alpha=0.8)
        axes[0].set_xlabel('Trajectory数量', fontsize=11)
        axes[0].set_title('各动作类型的Trajectory数量', fontsize=12, fontweight='bold')
        axes[0].grid(axis='x', alpha=0.3)
        
        # 图2：样本数量
        axes[1].barh(self.stats['动作类型'], self.stats['样本数量'], 
                     color='coral', alpha=0.8)
        axes[1].set_xlabel('样本数量', fontsize=11)
        axes[1].set_title('各动作类型的样本数量', fontsize=12, fontweight='bold')
        axes[1].grid(axis='x', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def get_dataframe(self):
        """返回统计DataFrame"""
        return self.stats
    
    def export_to_csv(self, output_path='action_statistics.csv'):
        """导出统计结果到CSV"""
        if self.stats is None:
            print("请先运行 analyze() 方法")
            return
        
        self.stats.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✓ 统计结果已保存到: {output_path}")


# ==================== 使用示例 ====================

# 1. 创建分析器并加载数据
analyzer = ActionAnalyzer('/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_frm.jsonl')  # 替换为你的文件路径
analyzer.load_data()

# 2. 执行分析
analyzer.analyze()

# 3. 打印统计摘要
analyzer.print_summary()


✓ 成功加载 16359 条数据

📊 动作类型统计分析

总计：
  • 动作类型数量: 23
  • Trajectory总数: 957
  • 样本总数: 16359

详细统计：
                                                         动作类型  Trajectory数量  样本数量
                                cylinder_pick_box_place_close            56   954
                                                   gear_place            48   945
                                          throw_battery_twice            45   942
                                                    nut_place            45   894
                                      place_button_then_press            50   888
                                                throw_battery            48   873
                               battery_insertion_with_pullout            49   843
                                      plug_pullout_then_press            46   828
                                                seal_stamping            42   804
                                            plug_insertion_v2            43   786
    

In [4]:
import os
import json
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
# 输入与输出
INPUT_JSONL = '/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/annotation/h5_tienkung_xsens_frm.jsonl'  # 替换为你的文件路径
OUTPUT_DIR  = './splits'  # 输出目录（会自动创建）
TRAIN_JSONL = os.path.join(OUTPUT_DIR, 'train.jsonl')
TEST_JSONL  = os.path.join(OUTPUT_DIR, 'test.jsonl')

# 固定测试集 5 个类别（其余全部作为训练）
TEST_CLASSES = {
    'brick_piled_then_press_thrice',
    'plug_pullout_then_press',
    'switch_manipulation',
    'pour_bread_then_place',
    'throw_battery_twice',
}
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    """
    从ID中提取动作类型
    约定: 格式为 数据集名称/动作类型/时间戳 或相似（至少包含两段）
    """
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def split_by_classes(data, test_classes: set):
    """根据类别集合划分 train/test（类别互斥）"""
    train, test = [], []
    for item in data:
        act = extract_action_type(item.get('id', ''))
        if act in test_classes:
            test.append(item)
        else:
            train.append(item)
    return train, test

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def summarize_split(data, title: str):
    """输出该 split 内的基础统计（样本/trajectory/按类别分布）"""
    # 统计
    action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
    for item in data:
        id_string = item.get('id', '')
        action_type = extract_action_type(id_string)
        if action_type:
            # 按你的原始逻辑：以完整 id 作为 trajectory 唯一标识
            action_data[action_type]['trajectories'].add(id_string)
            action_data[action_type]['samples'] += 1

    # 转 DataFrame
    rows = []
    for act, info in action_data.items():
        rows.append({
            '动作类型': act,
            'Trajectory数量': len(info['trajectories']),
            '样本数量': info['samples']
        })
    df = pd.DataFrame(rows).sort_values('样本数量', ascending=False)

    # 汇总
    n_actions = len(df)
    n_traj = int(df['Trajectory数量'].sum()) if not df.empty else 0
    n_samples = int(df['样本数量'].sum()) if not df.empty else 0

    print("\n" + "="*60)
    print(f"📊 {title} 统计")
    print("="*60)
    print(f"  • 动作类型数量: {n_actions}")
    print(f"  • Trajectory总数: {n_traj}")
    print(f"  • 样本总数: {n_samples}")
    if not df.empty:
        print("\n详细统计：")
        print(df.to_string(index=False))
    else:
        print("\n该 split 为空。")
    print("="*60 + "\n")

    return {
        'n_actions': n_actions,
        'n_traj': n_traj,
        'n_samples': n_samples,
        'df': df
    }

def main():
    ensure_dir(OUTPUT_DIR)

    # 1) 读入
    data = load_jsonl(INPUT_JSONL)

    # 2) 切分（类别互斥）
    train, test = split_by_classes(data, TEST_CLASSES)

    # 3) 写出
    dump_jsonl(train, TRAIN_JSONL)
    dump_jsonl(test, TEST_JSONL)
    print(f"✓ 已写出 Train: {len(train)} 条 -> {TRAIN_JSONL}")
    print(f"✓ 已写出 Test : {len(test)} 条 -> {TEST_JSONL}")

    # 4) 统计（分别对 Train/Test）
    train_stat = summarize_split(train, "Train")
    test_stat  = summarize_split(test,  "Test")

    # 5) 汇总比例（便于快速 sanity check）
    total_samples = train_stat['n_samples'] + test_stat['n_samples']
    total_traj    = train_stat['n_traj'] + test_stat['n_traj']
    total_actions = train_stat['n_actions'] + test_stat['n_actions']  # 应为全体类别数量

    print("📌 整体占比（按样本/trajectory/类别数）")
    if total_samples > 0:
        print(f"  • Test 样本占比: {test_stat['n_samples']}/{total_samples} = {test_stat['n_samples']/total_samples:.2%}")
    if total_traj > 0:
        print(f"  • Test Trajectory占比: {test_stat['n_traj']}/{total_traj} = {test_stat['n_traj']/total_traj:.2%}")
    if total_actions > 0:
        print(f"  • Test 类别占比: {test_stat['n_actions']}/{total_actions} = {test_stat['n_actions']/total_actions:.2%}")

    print("\n✅ 切分完成。类别互斥：Test = 5 类；Train = 其余全部类别。")

if __name__ == '__main__':
    main()


✓ 成功加载 16359 条数据
✓ 已写出 Train: 12849 条 -> ./splits/train.jsonl
✓ 已写出 Test : 3510 条 -> ./splits/test.jsonl

📊 Train 统计
  • 动作类型数量: 18
  • Trajectory总数: 761
  • 样本总数: 12849

详细统计：
                                                         动作类型  Trajectory数量  样本数量
                                cylinder_pick_box_place_close            56   954
                                                   gear_place            48   945
                                                    nut_place            45   894
                                      place_button_then_press            50   888
                                                throw_battery            48   873
                               battery_insertion_with_pullout            49   843
                                                seal_stamping            42   804
                                            plug_insertion_v2            43   786
                                                   wipe_panel            48   741
   

In [1]:
import os
import json
import random
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
INPUT_JSONL = '/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/progrsslm/annotation/h5_tienkung_xsens_frm/h5_tienkung_xsens_frm_train.jsonl'
OUTPUT_DIR  = '/home/runsheng/personal_3/qiancx/Sources/datasets/robomind/progrsslm/annotation/h5_tienkung_xsens_frm'
SFT_JSONL   = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_sft.jsonl')
RL_JSONL    = os.path.join(OUTPUT_DIR, 'h5_tienkung_xsens_rl.jsonl')
SPLIT_RATIO = 0.2   # SFT 占 20%，RL 占 80%
SEED = 42
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def group_by_trajectory(data):
    """
    将数据按完整 id（即轨迹）聚合。
    返回: {action_type: {trajectory_id: [items...] } }
    """
    grouped = defaultdict(lambda: defaultdict(list))
    for item in data:
        tid = item.get('id', '')
        act = extract_action_type(tid)
        if act:
            grouped[act][tid].append(item)
    return grouped

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def main():
    random.seed(SEED)
    ensure_dir(OUTPUT_DIR)

    # 1) 加载数据
    data = load_jsonl(INPUT_JSONL)

    # 2) 按类别 & 轨迹分组
    grouped = group_by_trajectory(data)
    print(f"✓ 共发现 {len(grouped)} 个类别（基于 extract_action_type）\n")

    # 3) 分层划分 SFT / RL
    sft_data, rl_data = [], []

    category_stats = []  # 用于打印统计
    for act, traj_map in grouped.items():
        traj_ids = list(traj_map.keys())
        random.shuffle(traj_ids)

        # 计算 SFT 轨迹数量
        sft_count = int(len(traj_ids) * SPLIT_RATIO)
        sft_traj = traj_ids[:sft_count]
        rl_traj = traj_ids[sft_count:]

        # 收集数据
        sft_items = [item for tid in sft_traj for item in traj_map[tid]]
        rl_items = [item for tid in rl_traj for item in traj_map[tid]]

        sft_data.extend(sft_items)
        rl_data.extend(rl_items)

        # 记录统计
        category_stats.append({
            '类别': act,
            'SFT_轨迹数': len(sft_traj),
            'RL_轨迹数': len(rl_traj),
            'SFT_样本数': len(sft_items),
            'RL_样本数': len(rl_items),
        })

    # 4) 写出文件
    dump_jsonl(sft_data, SFT_JSONL)
    dump_jsonl(rl_data, RL_JSONL)

    # 5) 打印统计信息
    print("\n====== 数据集划分完成 ======")
    print(f"总类别数: {len(category_stats)}")
    print(f"SFT : RL = {SPLIT_RATIO:.0%} : {1-SPLIT_RATIO:.0%}\n")

    print("类别分布统计:")
    print("------------------------------------------------------------")
    print(f"{'类别':25s} | {'SFT轨迹':7s} {'RL轨迹':7s} {'SFT样本':7s} {'RL样本':7s}")
    print("------------------------------------------------------------")
    for stat in category_stats:
        print(f"{stat['类别']:25s} | "
              f"{stat['SFT_轨迹数']:7d} {stat['RL_轨迹数']:7d} "
              f"{stat['SFT_样本数']:7d} {stat['RL_样本数']:7d}")
    print("------------------------------------------------------------")
    print(f"SFT 总样本: {len(sft_data)}   RL 总样本: {len(rl_data)}")
    print(f"👉 输出文件已生成:\n   - {SFT_JSONL}\n   - {RL_JSONL}\n")

if __name__ == '__main__':
    main()


✓ 成功加载 12849 条数据
✓ 共发现 18 个类别（基于 extract_action_type）


====== 数据集划分完成 ======
总类别数: 18
SFT : RL = 20% : 80%

类别分布统计:
------------------------------------------------------------
类别                        | SFT轨迹   RL轨迹    SFT样本   RL样本   
------------------------------------------------------------
battery_insertion_with_pullout |       9      40     162     681
cylinder_pick_box_place_close |      11      45     198     756
pick_shelf_insert_machine_press_switch_place_plate |       6      27     108     477
plate_push                |       9      37      84     327
place_button              |       9      36     135     537
plug_extract_from         |       7      29     129     516
place_button_then_press   |      10      40     186     702
plug_pullout              |       6      25     117     462
seal_stamping             |       8      34     147     657
nut_place                 |       9      36     180     714
plug_insertion_v2         |       8      35     141     645
plug_in

In [1]:
import os
import json
import random
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
INPUT_JSONL = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/text_demo/h5_tienkung_xsens_text_all.jsonl'
OUTPUT_DIR  = '/home/vcj9002/jianshu/workspace/code/ProgressLM/data/train/text_demo'
SFT_JSONL   = os.path.join(OUTPUT_DIR, 'text_h5_tienkung_xsens_sft.jsonl')
RL_JSONL    = os.path.join(OUTPUT_DIR, 'text_h5_tienkung_xsens_rl.jsonl')
SPLIT_RATIO = 0.2   # SFT 占 20%，RL 占 80%
SEED = 42
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def group_by_trajectory(data):
    """
    将数据按完整 id（即轨迹）聚合。
    返回: {action_type: {trajectory_id: [items...] } }
    """
    grouped = defaultdict(lambda: defaultdict(list))
    for item in data:
        tid = item.get('id', '')
        act = extract_action_type(tid)
        if act:
            grouped[act][tid].append(item)
    return grouped

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def main():
    random.seed(SEED)
    ensure_dir(OUTPUT_DIR)

    # 1) 加载数据
    data = load_jsonl(INPUT_JSONL)

    # 2) 按类别 & 轨迹分组
    grouped = group_by_trajectory(data)
    print(f"✓ 共发现 {len(grouped)} 个类别（基于 extract_action_type）\n")

    # 3) 分层划分 SFT / RL
    sft_data, rl_data = [], []

    category_stats = []  # 用于打印统计
    for act, traj_map in grouped.items():
        traj_ids = list(traj_map.keys())
        random.shuffle(traj_ids)

        # 计算 SFT 轨迹数量
        sft_count = int(len(traj_ids) * SPLIT_RATIO)
        sft_traj = traj_ids[:sft_count]
        rl_traj = traj_ids[sft_count:]

        # 收集数据
        sft_items = [item for tid in sft_traj for item in traj_map[tid]]
        rl_items = [item for tid in rl_traj for item in traj_map[tid]]

        sft_data.extend(sft_items)
        rl_data.extend(rl_items)

        # 记录统计
        category_stats.append({
            '类别': act,
            'SFT_轨迹数': len(sft_traj),
            'RL_轨迹数': len(rl_traj),
            'SFT_样本数': len(sft_items),
            'RL_样本数': len(rl_items),
        })

    # 4) 写出文件
    dump_jsonl(sft_data, SFT_JSONL)
    dump_jsonl(rl_data, RL_JSONL)

    # 5) 打印统计信息
    print("\n====== 数据集划分完成 ======")
    print(f"总类别数: {len(category_stats)}")
    print(f"SFT : RL = {SPLIT_RATIO:.0%} : {1-SPLIT_RATIO:.0%}\n")

    print("类别分布统计:")
    print("------------------------------------------------------------")
    print(f"{'类别':25s} | {'SFT轨迹':7s} {'RL轨迹':7s} {'SFT样本':7s} {'RL样本':7s}")
    print("------------------------------------------------------------")
    for stat in category_stats:
        print(f"{stat['类别']:25s} | "
              f"{stat['SFT_轨迹数']:7d} {stat['RL_轨迹数']:7d} "
              f"{stat['SFT_样本数']:7d} {stat['RL_样本数']:7d}")
    print("------------------------------------------------------------")
    print(f"SFT 总样本: {len(sft_data)}   RL 总样本: {len(rl_data)}")
    print(f"👉 输出文件已生成:\n   - {SFT_JSONL}\n   - {RL_JSONL}\n")

if __name__ == '__main__':
    main()


✓ 成功加载 5919 条数据
✓ 共发现 24 个类别（基于 extract_action_type）


====== 数据集划分完成 ======
总类别数: 24
SFT : RL = 20% : 80%

类别分布统计:
------------------------------------------------------------
类别                        | SFT轨迹   RL轨迹    SFT样本   RL样本   
------------------------------------------------------------
battery_insertion_with_pullout |      11      46      66     262
cylinder_pick_box_place_close |      11      47      60     269
brick_piled_then_press_thrice |       8      35      44     187
pick_shelf_insert_machine_press_switch_place_plate |       7      30      40     178
plate_push                |       8      35      27     108
place_button              |       9      39      49     187
plug_extract_from         |       8      32      44     194
plug_pullout_then_press   |       9      38      56     225
pour_bread_then_place     |       7      30      43     170
place_button_then_press   |      11      44      61     260
plug_pullout              |       7      29      44     183
seal

In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict

# ===================== 配置区 =====================
# 输入与输出
INPUT_JSONL = '/Users/cxqian/Codes/ProgressLM/data/train/text_demo/text_h5_tienkung_xsens_sft.jsonl'  # 替换为你的文件路径
OUTPUT_DIR  = '/Users/cxqian/Codes/ProgressLM/data/eval/text'  # 输出目录（会自动创建）
TRAIN_JSONL = os.path.join(OUTPUT_DIR, 'text_h5_tienkung_xsens_sft')
TEST_JSONL  = os.path.join(OUTPUT_DIR, 'text_h5_tienkung_xsens_test2')

# 固定测试集 5 个类别（其余全部作为训练）
TEST_CLASSES = {
    'brick_piled_then_press_thrice',
    'plug_pullout_then_press',
    'switch_manipulation',
    'pour_bread_then_place',
    'throw_battery_twice',
}
# =================================================

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def extract_action_type(id_string: str):
    """
    从ID中提取动作类型
    约定: 格式为 数据集名称/动作类型/时间戳 或相似（至少包含两段）
    """
    parts = id_string.split('/')
    if len(parts) >= 2:
        return parts[1]
    return None

def load_jsonl(jsonl_path: str):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    print(f"✓ 成功加载 {len(data)} 条数据")
    return data

def split_by_classes(data, test_classes: set):
    """根据类别集合划分 train/test（类别互斥）"""
    train, test = [], []
    for item in data:
        act = extract_action_type(item.get('id', ''))
        if act in test_classes:
            test.append(item)
        else:
            train.append(item)
    return train, test

def dump_jsonl(data, out_path: str):
    with open(out_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def summarize_split(data, title: str):
    """输出该 split 内的基础统计（样本/trajectory/按类别分布）"""
    # 统计
    action_data = defaultdict(lambda: {'trajectories': set(), 'samples': 0})
    for item in data:
        id_string = item.get('id', '')
        action_type = extract_action_type(id_string)
        if action_type:
            # 按你的原始逻辑：以完整 id 作为 trajectory 唯一标识
            action_data[action_type]['trajectories'].add(id_string)
            action_data[action_type]['samples'] += 1

    # 转 DataFrame
    rows = []
    for act, info in action_data.items():
        rows.append({
            '动作类型': act,
            'Trajectory数量': len(info['trajectories']),
            '样本数量': info['samples']
        })
    df = pd.DataFrame(rows).sort_values('样本数量', ascending=False)

    # 汇总
    n_actions = len(df)
    n_traj = int(df['Trajectory数量'].sum()) if not df.empty else 0
    n_samples = int(df['样本数量'].sum()) if not df.empty else 0

    print("\n" + "="*60)
    print(f"📊 {title} 统计")
    print("="*60)
    print(f"  • 动作类型数量: {n_actions}")
    print(f"  • Trajectory总数: {n_traj}")
    print(f"  • 样本总数: {n_samples}")
    if not df.empty:
        print("\n详细统计：")
        print(df.to_string(index=False))
    else:
        print("\n该 split 为空。")
    print("="*60 + "\n")

    return {
        'n_actions': n_actions,
        'n_traj': n_traj,
        'n_samples': n_samples,
        'df': df
    }

def main():
    ensure_dir(OUTPUT_DIR)

    # 1) 读入
    data = load_jsonl(INPUT_JSONL)

    # 2) 切分（类别互斥）
    train, test = split_by_classes(data, TEST_CLASSES)

    # 3) 写出
    dump_jsonl(train, TRAIN_JSONL)
    dump_jsonl(test, TEST_JSONL)
    print(f"✓ 已写出 Train: {len(train)} 条 -> {TRAIN_JSONL}")
    print(f"✓ 已写出 Test : {len(test)} 条 -> {TEST_JSONL}")

    # 4) 统计（分别对 Train/Test）
    train_stat = summarize_split(train, "Train")
    test_stat  = summarize_split(test,  "Test")

    # 5) 汇总比例（便于快速 sanity check）
    total_samples = train_stat['n_samples'] + test_stat['n_samples']
    total_traj    = train_stat['n_traj'] + test_stat['n_traj']
    total_actions = train_stat['n_actions'] + test_stat['n_actions']  # 应为全体类别数量

    print("📌 整体占比（按样本/trajectory/类别数）")
    if total_samples > 0:
        print(f"  • Test 样本占比: {test_stat['n_samples']}/{total_samples} = {test_stat['n_samples']/total_samples:.2%}")
    if total_traj > 0:
        print(f"  • Test Trajectory占比: {test_stat['n_traj']}/{total_traj} = {test_stat['n_traj']/total_traj:.2%}")
    if total_actions > 0:
        print(f"  • Test 类别占比: {test_stat['n_actions']}/{total_actions} = {test_stat['n_actions']/total_actions:.2%}")

    print("\n✅ 切分完成。类别互斥：Test = 5 类；Train = 其余全部类别。")

if __name__ == '__main__':
    main()


✓ 成功加载 4376 条数据
✓ 已写出 Train: 3436 条 -> /Users/cxqian/Codes/ProgressLM/data/eval/text/text_h5_tienkung_xsens_rl
✓ 已写出 Test : 940 条 -> /Users/cxqian/Codes/ProgressLM/data/eval/text/text_h5_tienkung_xsens_test

📊 Train 统计
  • 动作类型数量: 18
  • Trajectory总数: 608
  • 样本总数: 3436

详细统计：
                                                         动作类型  Trajectory数量  样本数量
                                                   gear_place            40   263
                                cylinder_pick_box_place_close            45   258
                                                    nut_place            38   254
                                      place_button_then_press            38   233
                                                throw_battery            39   233
                               battery_insertion_with_pullout            38   215
                                                seal_stamping            33   208
                                                   wipe_panel     